In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
from tensorflow.keras.utils import to_categorical

In [ ]:
cols = ['variance ','skewness','curtosis','entropy', 'class']
df = pd.read_csv('data_banknote_authentication.txt', names=cols, index_col=False) 
df

,variance,skewness,curtosis,entropy,class
0,3.62160,8.66610,-2.8073,-0.44699,0
1,4.54590,8.16740,-2.4586,-1.46210,0
2,3.86600,-2.63830,1.9242,0.10645,0
3,3.45660,9.52280,-4.0112,-3.59440,0
4,0.32924,-4.45520,4.5718,-0.98880,0
...,...,...,...,...,...
1367,0.40614,1.34920,-1.4501,-0.55949,1
1368,-1.38870,-4.87730,6.4774,0.34179,1
1369,-3.75030,-13.45860,17.5932,-2.77710,1
1370,-3.56370,-8.38270,12.3930,-1.28230,1


In [ ]:
y = df['class']
x = df.drop('class',axis=1)
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.33, random_state = 1)

In [ ]:


activation = ['relu', 'softmax', 'tanh']
optimizer  = ['adam', 'SGD'] 
loss = ['binary_crossentropy','hinge', 'squared_hinge']
batch_size = [15, 32, 100]


t = pd.DataFrame(columns=[
    'activation', 'optimizer', 'loss', 'batch_size',
    'train score','test score' 
])




for a in activation:
  for o in optimizer:
    for l in loss:
      for bs in batch_size:
        model = Sequential()
        model.add(Dense(6, activation=a, input_dim=train_x.shape[1]))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer=o, loss=l,metrics=['binary_accuracy'])
        model.fit(train_x, train_y, validation_split=0.2, epochs=10, batch_size=bs, verbose=0)

        t.loc[t.shape[0]] = [a,o,l,bs,model.evaluate(train_x, train_y)[1],model.evaluate(test_x, test_y)[1]]

t.head()

15/15 [==============================] - 0s 2ms/step - loss: 0.8113 - binary_accuracy: 0.9338


,activation,optimizer,loss,batch_size,train score,test score
0,relu,adam,binary_crossentropy,15,0.957563,0.951435
1,relu,adam,binary_crossentropy,32,0.724701,0.721854
2,relu,adam,binary_crossentropy,100,0.546246,0.565121
3,relu,adam,hinge,15,0.845484,0.878587
4,relu,adam,hinge,32,0.820457,0.836645


In [ ]:
t = t.sort_values('test score', ascending=False)
t.head(10)

,activation,optimizer,loss,batch_size,train score,test score
15,relu,SGD,squared_hinge,15,0.980413,0.984547
6,relu,adam,squared_hinge,15,0.973885,0.984547
9,relu,SGD,binary_crossentropy,15,0.978237,0.980132
45,tanh,SGD,binary_crossentropy,15,0.980413,0.964680
10,relu,SGD,binary_crossentropy,32,0.930359,0.960265
0,relu,adam,binary_crossentropy,15,0.957563,0.951435
48,tanh,SGD,hinge,15,0.952122,0.951435
21,softmax,adam,hinge,15,0.959739,0.951435
39,tanh,adam,hinge,15,0.948857,0.947020
36,tanh,adam,binary_crossentropy,15,0.946681,0.947020


In [ ]:
model = Sequential()
       
model.add(Dense(6, activation=t.iloc[0]['activation'], input_dim=train_x.shape[1]))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=t.iloc[0]['optimizer'], loss=t.iloc[0]['loss'],metrics=['binary_accuracy'])
history = model.fit(train_x, train_y, validation_split=0.2, epochs=10, batch_size=t.iloc[0]['batch_size'], verbose=0)

model.save('myModel')
model = keras.models.load_model('myModel')
model.summary()

Model: "sequential_277"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_660 (Dense)           (None, 6)                 30        
                                                                 
 dense_661 (Dense)           (None, 1)                 7         
                                                                 
Total params: 37
Trainable params: 37
Non-trainable params: 0
_________________________________________________________________
